## Welcome to Talkwalker-inator v2

#### Import!

In [1]:
!pip install selenium beautifulsoup4
!apt-get update
!apt install chromium-chromedriver
!pip install selenium-screenshot
!pip install webdriver-manager
!pip install fuzzywuzzy
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.3 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,326 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,599 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/uni

#### Install necessary libraries



In [2]:
#data scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

#data handling
import urllib.request
import pandas as pd
import json
import time
import re
import datetime
import numpy as np
import openpyxl
import os
from IPython.display import FileLink
from fuzzywuzzy import fuzz
import random

#### Intialize Chrome WebDriver

In [3]:
# Initialize the Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-infobars")
browser = webdriver.Chrome(options=chrome_options)

#### Log In

In [4]:
with open('facebook_credentials.txt') as file:
    EMAIL = file.readline().split('"')[1]
    PASSWORD = file.readline().split('"')[1]

browser.get("http://facebook.com")
browser.maximize_window()
wait = WebDriverWait(browser, 30)
email_field = wait.until(EC.visibility_of_element_located((By.NAME, 'email')))
email_field.send_keys(EMAIL)
pass_field = wait.until(EC.visibility_of_element_located((By.NAME, 'pass')))
pass_field.send_keys(PASSWORD)
pass_field.send_keys(Keys.RETURN)

time.sleep(5)

#### Data Ingestion

In [25]:
search_queries = [
    'Primecare Physical Therapy',
    'Buela Physical Therapy',
    'ACC Physical Therapy Clinic - Imus',
    'ReAbility Physical Therapy - Imus',
    'Homecare Physical Therapy',
    'FortMED Medical Clinics',
    'Affinity Dental Clinics Alabang',
    'Wellcare Clinics & Lab.',
    'St Bridget and Roch Clinic',
    'Dr. Beverly Ong-Amoranto Dermatology Clinics',
    'Notre Dame Medico Dental Clinics, Inc.',
    'PrimeHealth Medical Clinics and Laboratory',
    'Dr. James Dental Clinic',
    'Marlea Dental Clinic',
    'HSG - Health So Good Clinics and Diagnostics Center',
    'Smile More Dental Clinic',
    'Acain - Garde Dental Clinic',
    'Tooth Options Dental Clinic',
    'Iconic Smile Dental Clinic',
    'Telemedicine PH: Urology & Multispecialty Clinic',
    'Avignon Clinic',
    'MV Santiago Medical Center and Orthopedic Hospital Inc.',
    'Pagsanjan Medical Clinic Inc & General Hospital',
    'Unihealth - Sta. Rosa Hospital and Medical Center, Inc.',
    'Carmona Hospital and Medical Center',
    'Los Baños Doctors Hospital and Medical Center',
    'First Cabuyao Hospital and Medical Center',
    'Manila Doctors Hospital',
    'The Eye MD Ophthalmology Clinic',
    'Adorable Teeth Dental Clinic',
    'Affinity Dental Clinics Alabang',
    'Tooth & Go dental clinic',
    'Novodental U.P. Town Center',
    'DRA. RONA ORTHO-DENTAL CLINIC',
    'Smile Perfection Dental Clinic Philippines',
    'Tooth Options Dental Clinic',
    'CorDerm Advanced Dermatology and Laser Center',
    'K Dermatology Skin Clinic and Laser Center - Dr. Kat Abat-Pablo',
    'Skin Station',
    'Derma 360',
    'Skin Clean Dermatology Clinic',
    'YSA Dermatology-Dentistry BGC',
    'D V Bautista Dermatology Clinic & Laser Center',
    'HOO Dermatology',
    'Wink Laser Studio',
    'Naito Clinic',
    'Physio Point Physical Therapy And Rehab Medicine Clinic Sta Rosa',
    'Prohealth Sports And Spinal Physiotherapy Centres Philippines',
    'Hi Physiotherapy Clinic - San Pedro Laguna',
    'Olano Mabalay Pain and Wellness Physical Therapy Clinic',
    'Dynamic Grit Physical Therapy Clinic',
    'Summit Team Wellness Center',
    'Health Hub Pro Clinic Banawe Quezon City',
    "The Executive Facial Care",
    'Omron Healthcare',
    'Maxicare Healthcare Corporation',
    'Batangas Healthcare Specialists Medical Center',
    'Intramed Healthcare Products Inc.',
    'PhilHealth',
    'Skin 101',
    'MediCard Philippines Inc.',
    'Intellicare',
    'Value Care Health Systems, Inc.',
    'Caritas Health Shield',
    'Aon',
    'Pacific Cross Philippines (former Blue Cross Health Care)',
    'East West Healthcare Phils.',
    'Fortune Medicare',
    'HMI',
    'Getwell Health Systems Inc.'
]

#### Scraping Engine

In [26]:
# Function to get data using JavaScript XPath
def get_data_by_xpath(browser, xpath):
    return browser.execute_script(xpath)

# Function to scrape data from a Facebook page
def scrape_page_data(browser, search_query, df):
    # Construct the URL
    url = f"https://www.facebook.com/search/pages/?q={search_query}"
    browser.get(url)

    # Click the first search result
    try:
        result_xpath = """
            var element = document.evaluate('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div[2]/div[1]/div/div/div[1]/span/div/a', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            return element ? element.href : null;
        """
        result_url = get_data_by_xpath(browser, result_xpath)
        browser.get(result_url)

        # Wait for the page to load
        time.sleep(5)

        # Obtain data
        likes_xpath = """
            var element = document.evaluate('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div[3]/div/div/div[2]/span/a[1]/text()[1]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            return element ? element.textContent : null;
        """
        followers_xpath = """
            var element = document.evaluate('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div[3]/div/div/div[2]/span/a[2]/text()[1]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            return element ? element.textContent : null;
        """
        fb_page_name_xpath = """
            var element = document.evaluate('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div[3]/div/div/div[1]/div/div/span/h1/text()[1]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            return element ? element.textContent : null;
        """
        type_xpath = """
            var element = document.evaluate('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div[3]/div/div/div[1]/div/div/span', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            return element ? element.textContent : null;
        """

        likes = get_data_by_xpath(browser, likes_xpath)
        followers = get_data_by_xpath(browser, followers_xpath)
        fb_page_name = get_data_by_xpath(browser, fb_page_name_xpath)
        page_type = get_data_by_xpath(browser, type_xpath)
        current_url = browser.current_url

        # Append data to the dataframe
        df = df.append({
            'Search_Query': search_query,
            'FB_Page_Name': fb_page_name,
            'Likes': likes,
            'Followers': followers,
            'Type': page_type,
            'Current_URL': current_url
        }, ignore_index=True)

    except TimeoutException:
        print(f"Timeout exception for search query: {search_query}")

    return df

# Load existing data from a CSV file if it exists
try:
    result_df = pd.read_excel('facebook_data.xlsx')
except FileNotFoundError:
    # Create an empty dataframe if the file does not exist
    columns = ['Search_Query', 'FB_Page_Name', 'Likes', 'Followers', 'Type', 'Current_URL']
    result_df = pd.DataFrame(columns=columns)

# Iterate through search queries
for query in search_queries:
    try:
        result_df = scrape_page_data(browser, query, result_df)

        # Introduce a random sleep duration between 4 and 6 seconds
        sleep_duration = random.uniform(4, 6)
        time.sleep(sleep_duration)

    except Exception as e:
        print(f"Error processing query {query}: {e}")
        continue

# Save the updated dataframe to a CSV file
result_df.to_excel('facebook_data.xlsx', index=False)


<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame

Error processing query Marlea Dental Clinic: name 'TimeoutException' is not defined


<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-26-9e34de3ea953>:48: FutureWarning: The frame

In [27]:
# Display the resulting dataframe
result_df.head()

,Search_Query,FB_Page_Name,Likes,Followers,Type,Current_URL
0,Primecare Physical Therapy,Primecare Physical Therapy,650,785,Primecare Physical Therapy,https://www.facebook.com/primecare.qc
1,Buela Physical Therapy,Buela Physical Therapy,17K,18K,Buela Physical Therapy,https://www.facebook.com/BuelaPT
2,ACC Physical Therapy Clinic - Imus,ACC Physical Therapy Clinic - Imus,443,647,ACC Physical Therapy Clinic - Imus,https://www.facebook.com/accptimus
3,ReAbility Physical Therapy - Imus,ReAbility Physical Therapy - Imus,4.1K,4.3K,ReAbility Physical Therapy - Imus,https://www.facebook.com/ReAbility.Imus
4,Homecare Physical Therapy,Physical Therapy Session,41K,1M,Physical Therapy Session,https://www.facebook.com/nihaymariwa


In [28]:
# Save the updated dataframe to a CSV file
result_df.to_excel('facebook_data.xlsx', index=False)
result_df = pd.read_excel('facebook_data.xlsx')

#### Post processing

In [29]:
result_df = pd.read_excel('facebook_data.xlsx')
result_df = result_df.drop_duplicates(subset='FB_Page_Name', keep='first').reset_index(drop=True)

In [30]:
def update_type(value):
    if pd.notna(value) and "Verified account" in value:
        return "Verified account"
    else:
        return "Unverified account"

# Update values in the 'Type' column
result_df['Type'] = result_df['Type'].apply(update_type)

In [31]:
def convert_to_numeric(value):
    if pd.notna(value):
        if isinstance(value, str):
            if 'k' in value.lower():
                return int(float(value.lower().replace('k', '')) * 1000)
            elif 'm' in value.lower():
                return int(float(value.lower().replace('m', '')) * 1000000)
            else:
                return int(value)
        else:
            return int(value)

# Create new numeric columns 'Likes_Numeric' and 'Followers_Numeric'
result_df['Likes_Numeric'] = result_df['Likes'].apply(convert_to_numeric)
result_df['Followers_Numeric'] = result_df['Followers'].apply(convert_to_numeric)

In [32]:
result_df.head(15)

,Search_Query,FB_Page_Name,Likes,Followers,Type,Current_URL,Likes_Numeric,Followers_Numeric
0,Primecare Physical Therapy,Primecare Physical Therapy,650,785,Unverified account,https://www.facebook.com/primecare.qc,650.0,785.0
1,Buela Physical Therapy,Buela Physical Therapy,17K,18K,Unverified account,https://www.facebook.com/BuelaPT,17000.0,18000.0
2,ACC Physical Therapy Clinic - Imus,ACC Physical Therapy Clinic - Imus,443,647,Unverified account,https://www.facebook.com/accptimus,443.0,647.0
3,ReAbility Physical Therapy - Imus,ReAbility Physical Therapy - Imus,4.1K,4.3K,Unverified account,https://www.facebook.com/ReAbility.Imus,4100.0,4300.0
4,Homecare Physical Therapy,Physical Therapy Session,41K,1M,Unverified account,https://www.facebook.com/nihaymariwa,41000.0,1000000.0
5,FortMED Medical Clinics,FortMED Medical Clinics,3.4K,3.6K,Unverified account,https://www.facebook.com/fortmed,3400.0,3600.0
6,Affinity Dental Clinics Alabang,Affinity Dental Clinics Alabang,19K,19K,Unverified account,https://www.facebook.com/affinitydental.alaban...,19000.0,19000.0
7,Wellcare Clinics & Lab.,Wellcare Clinics,1.8K,2K,Unverified account,https://www.facebook.com/WellCareClinic,1800.0,2000.0
8,St Bridget and Roch Clinic,St. Bridget Children's Clinic,1.4K,1.5K,Unverified account,https://www.facebook.com/SheiannaMae,1400.0,1500.0
9,Dr. Beverly Ong-Amoranto Dermatology Clinics,Dr. Beverly Ong-Amoranto Dermatology Clinics,1.6K,1,Unverified account,https://www.facebook.com/DrBebsO.Amoranto,1600.0,1.0


In [33]:
result_df.to_excel('facebook_data_processed.xlsx', index=False)

#### Quitting the Browser

In [34]:
# Close the browser
browser.quit()

#### Diagnostics

In [7]:
browser.save_screenshot("screenshot.png")

True

## Accessories

#### Scraping Engine

In [ ]:
for index, row in df.iterrows():
    url = row['Post URL']
    browser.get(url)
    current_url = browser.current_url
    time.sleep(5)

    if "watch" in current_url:
        js_path = '#watch_feed > div > div.x78zum5.xdt5ytf.x1huibft.x1n6yrxt > div > div > div.x1jx94hy.x78zum5.xdt5ytf.x2lah0s.xw2csxc.x1odjw0f.xmd6oqt.x13o0s5z > div.x1swvt13.x1pi30zi.xyamay9 > div:nth-child(5) > div > span'

        try:
            element = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, js_path))
            )
            caption = element.text
            df.at[index, 'Post_Caption'] = caption

        except TimeoutException:
            print(f"Timeout exception for URL: {url}")
            continue

    elif "posts" in current_url:
        js_code = """
        var element = document.evaluate('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[1]/div/div/div/span/div[1]/div', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
        return element ? element.textContent : null;
    """
        element_text = browser.execute_script(js_code)
        caption = element_text

        df.at[index, 'Post_Caption'] = caption

    elif "events" in current_url:
        js_code = """
        var element = document.evaluate('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div/div/div[6]/div/span', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
        return element ? element.textContent : null;
    """
        element_text = browser.execute_script(js_code)
        caption = element_text

        df.at[index, 'Post_Caption'] = caption

    else:
        df.at[index, 'Post_Caption'] = None
        continue


df.head()


In [ ]:
df.head()

#### Trim and Clean

In [ ]:
def trim_and_clean(text):
    if isinstance(text, str):
        cleaned_text = text.strip()
        cleaned_text = cleaned_text.replace('\n', ' ')

        return cleaned_text
    else:
        return str(text)

df['Cleaned Post Caption'] = df['Post_Caption'].astype(str).apply(trim_and_clean)

####Downloading the File

In [ ]:
df2 = df.copy()
df2.drop(columns=['Post_Caption'], inplace=True)
df2.rename(columns={'Cleaned Post Caption': 'Post_Caption'}, inplace=True)

In [ ]:
xlsx_filename = 'FB Post Captions.xlsx'
df2.to_excel(xlsx_filename, index=False)

#### Terminating the Browser

In [ ]:
browser.quit()

## Archived

#### Reading the file

In [ ]:
target_directory = "/content/"
files = os.listdir(target_directory)
keyword = "brief"
min_match_score = 80
file_format = ".xlsx"

def is_approximate_match(filename, keyword, min_match_score):
    return fuzz.partial_ratio(filename.lower(), keyword.lower()) >= min_match_score

matching_files = [f for f in files if is_approximate_match(f, keyword, min_match_score) and f.endswith(file_format)]

if len(matching_files) == 0:
    print("No matching file found. Please upload an XLSX file with a similar name to 'brief'.")
else:
    try:
        matching_file = matching_files[0]
        file_path = os.path.join(target_directory, matching_file)
        df = pd.read_excel(file_path)
        print(f"Found a matching file: {matching_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
df.head()

#### Amending the file

In [ ]:
df.dropna(subset=[df.columns[1]], inplace=True)
df = df.reset_index(drop=True)
df['Post_Caption'] = None

In [ ]:
df.head()

#### Extracting the embedded hyperlinks

In [ ]:
target_directory = "/content/"
keyword = "brief"
min_match_score = 80
file_format = ".xlsx"

def is_approximate_match(filename, keyword, min_match_score):
    return fuzz.partial_ratio(filename.lower(), keyword.lower()) >= min_match_score

matching_file = None
files = os.listdir(target_directory)

for file in files:
    if file.endswith(file_format) and is_approximate_match(file, keyword, min_match_score):
        matching_file = file
        break

if matching_file is None:
    print("No matching file found. Please upload an XLSX file with a similar name to 'brief'.")
else:
    try:
        file_path = os.path.join(target_directory, matching_file)
        wb = openpyxl.load_workbook(file_path)
        ws = wb['Sheet1']
        links = []

        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=2, max_col=2):
            cell = row[0]
            if cell.hyperlink is not None:
                links.append(cell.hyperlink.target)
            elif cell.value:
                links.append("No URL Available")
            else:
                links.append(None)

        links = [link for link in links if link is not None]
        df['Post URL'] = links
        print(f"Found a matching file: {matching_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
df.head()

#### Filling the null the values

In [ ]:
current_value_movie_name = None

for i, value in enumerate(df.iloc[:, 0]):
    if not pd.isna(value):
        current_value_movie_name = value
    else:
        df.iat[i, 0] = current_value_movie_name

current_value = None

for i, value in enumerate(df.iloc[:, 2]):
    if not pd.isna(value):
        current_value_movie_type = value
    else:
        df.iat[i, 2] = current_value_movie_type


In [ ]:
df.tail()

In [ ]:
df.head()